# Install Packages and Setup Variables


In [1]:
!pip install -q openai==1.59.8 cohere==5.13.8 tiktoken==0.8.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 21.6 MB/s eta 0:00:00


In [2]:
import os

# Set the "OPENAI_API_KEY" and "GOOGLE_API_KEY" in the Python environment. Will be used by OpenAI client later.

os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_API_KEY>"
os.environ["GOOGLE_API_KEY"] = "<YOUR_GOOGLE_API_KEY>"


# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')
# os.environ["GOOGLE_API_KEY"] =  userdata.get('Google_api_key')

In [3]:
# False: Generate the embedding for the dataset. (Associated cost with using OpenAI endpoint)
# True: Load the dataset that already has the embedding vectors.
load_embedding = False

# Load Dataset


## Download Dataset (JSON)


The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model.


In [4]:
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv

--2025-01-20 06:52:05--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173646 (170K) [text/plain]
Saving to: ‘mini-llama-articles.csv’

mini-llama-articles 100%[===================>] 169.58K  --.-KB/s    in 0.03s   

2025-01-20 06:52:05 (4.97 MB/s) - ‘mini-llama-articles.csv’ saved [173646/173646]

--2025-01-20 06:52:05--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP r

## Read File


In [5]:
# Split the input text into chunks of specified size.
def split_into_chunks(text, chunk_size=1024):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i : i + chunk_size])

    return chunks

In [6]:
import csv

chunks = []

# Load the file as a CSV
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
    csv_reader = csv.reader(file)

    for idx, row in enumerate(csv_reader):
        if idx == 0:
            continue
            # Skip header row
        chunks.extend(split_into_chunks(row[1]))

In [7]:
print("number of articles:", idx)
print("number of chunks:", len(chunks))

number of articles: 14
number of chunks: 174


In [8]:
import pandas as pd

# Convert the JSON list to a Pandas Dataframe
df = pd.DataFrame(chunks, columns=["chunk"])

df.keys()

Index(['chunk'], dtype='object')

# Generate Embedding


In [9]:
from openai import OpenAI

client = OpenAI()


# Defining a function that converts a text to embedding vector using OpenAI's Ada model.
def get_embedding(text):
    try:
        # Remove newlines
        text = text.replace("\n", " ")
        res = client.embeddings.create(input=[text], model="text-embedding-3-small")

        return res.data[0].embedding

    except:
        return None

In [10]:
from tqdm.notebook import tqdm
import numpy as np

# Generate embedding
if not load_embedding:
    print("Generating embeddings...")
    embeddings = []
    for index, row in tqdm(df.iterrows()):
        # df.at[index, 'embedding'] = get_embedding( row['chunk'] )
        embeddings.append(get_embedding(row["chunk"]))

    embeddings_values = pd.Series(embeddings)
    df.insert(loc=1, column="embedding", value=embeddings_values)

# Or, load the embedding from the file.
else:
    print("Loaded the embedding file.")
    # Load the file as a CSV
    df = pd.read_csv("mini-llama-articles-with_embeddings.csv")
    # Convert embedding column to an array
    df["embedding"] = df["embedding"].apply(lambda x: np.array(eval(x)), 0)

Generating embeddings...


0it [00:00, ?it/s]

In [11]:
# df.to_csv('mini-llama-articles-with_embeddings.csv')

# User Question


In [12]:
# Define the user question, and convert it to embedding.
QUESTION = "How many parameters LLaMA2 model has?"
QUESTION_emb = get_embedding(QUESTION)

len(QUESTION_emb)

1536

# Test Cosine Similarity


Calculating the similarity of embedding representations can help us to find pieces of text that are close to each other. In the following sample you see how the Cosine Similarity metric can identify which sentence could be a possible answer for the given user question. Obviously, the unrelated answer will score lower.


In [13]:
BAD_SOURCE_emb = get_embedding("The sky is blue.")
GOOD_SOURCE_emb = get_embedding("LLaMA2 model has a total of 2B parameters.")

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

# A sample that how a good piece of text can achieve high similarity score compared
# to a completely unrelated text.
print("> Bad Response Score:", cosine_similarity([QUESTION_emb], [BAD_SOURCE_emb]))
print("> Good Response Score:", cosine_similarity([QUESTION_emb], [GOOD_SOURCE_emb]))

> Bad Response Score: [[0.02578727]]
> Good Response Score: [[0.83154609]]


# Calculate Cosine Similarities


In [15]:
# The similarity between the questions and each part of the essay.
cosine_similarities = cosine_similarity([QUESTION_emb], df["embedding"].tolist())

print(cosine_similarities)

[[0.46767499 0.46912464 0.25982343 0.29374288 0.319654   0.40160312
  0.41500898 0.4525136  0.45935869 0.1259955  0.11750504 0.01347259
  0.22602134 0.21423916 0.10145219 0.33064027 0.1074138  0.346859
  0.16311856 0.08726645 0.3482437  0.22839007 0.19201346 0.26471736
  0.24928956 0.34824073 0.24828999 0.3276031  0.41445729 0.41337977
  0.46363194 0.38341214 0.46851769 0.35636739 0.35398223 0.3027117
  0.29934588 0.29252605 0.40029384 0.4646832  0.39473083 0.41039813
  0.4470362  0.43173664 0.35909244 0.33973082 0.51356637 0.20932135
  0.40203991 0.32829097 0.42834358 0.48270619 0.450364   0.34256287
  0.32083244 0.42588004 0.24622426 0.18089188 0.23648678 0.34272376
  0.3437286  0.20476358 0.1975709  0.22448873 0.21110849 0.42298903
  0.26382997 0.30438172 0.33609101 0.38368357 0.23536253 0.24351588
  0.37074498 0.28025883 0.49047132 0.53044055 0.37817696 0.43770825
  0.37750013 0.39255233 0.30081934 0.41710617 0.4674553  0.45420047
  0.35169137 0.21222866 0.4262131  0.31600177 0.440

In [16]:
import numpy as np

number_of_chunks_to_retrieve = 3

# Sort the scores
highest_index = np.argmax(cosine_similarities)

# Pick the N highest scored chunks
indices = np.argsort(cosine_similarities[0])[::-1][:number_of_chunks_to_retrieve]
print(indices)

[114  75  89]


In [17]:
# Look at the highest scored retrieved pieces of text
for idx, item in enumerate(df.chunk[indices]):
    print(f"> Chunk {idx+1}")
    print(item)
    print("----")

> Chunk 1
by Meta that ventures into both the AI and academic spaces. The model aims to help researchers, scientists, and engineers advance their work in exploring AI applications. It will be released under a non-commercial license to prevent misuse, and access will be granted to academic researchers, individuals, and organizations affiliated with the government, civil society, academia, and industry research facilities on a selective case-by-case basis. The sharing of codes and weights allows other researchers to test new approaches in LLMs. The LLaMA models have a range of 7 billion to 65 billion parameters. LLaMA-65B can be compared to DeepMind's Chinchilla and Google's PaLM. Publicly available unlabeled data was used to train these models, and training smaller foundational models require less computing power and resources. LLaMA 65B and 33B have been trained on 1.4 trillion tokens in 20 different languages, and according to the Facebook Artificial Intelligence Research (FAIR) team,

# Augment the Prompt


In [18]:
import google.generativeai as genai

# Use the Gemini API to answer the questions based on the retrieved pieces of text.
try:
    # Formulating the system prompt and condition the model to answer only AI-related questions.
    system_prompt = (
        "You are an assistant and expert in answering questions from a chunks of content. "
        "Only answer AI-related question, else say that you cannot answer this question."
    )

    # Create a user prompt with the user's question
    prompt = (
        "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
        "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )
    # Add the retrieved pieces of text to the prompt.
    prompt = prompt.format("".join(df.chunk[indices]), QUESTION)

    model = genai.GenerativeModel(
        model_name="gemini-1.5-flash", system_instruction=system_prompt
    )

    result = model.generate_content(
        prompt,
        request_options={"timeout": 1000},
    )
    res = result.text

except Exception as e:
    print(f"An error occurred: {e}")

In [19]:
print(res)

LLaMA 2 is available in four sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.



## How Augmenting the Prompt can address knowledge cutoff limitations and hallucinations

In [20]:
# Consider this as a retrieved chunk
# https://ai.meta.com/blog/meta-llama-3-1/
Example_chunk = """
Introducing Llama 3.1 Llama 3.1 405B is the first openly available model that rivals the top AI models when it comes to state-of-the-art capabilities in general knowledge, steerability, math, tool use, and multilingual translation. With the release of the 405B model, we’re poised to supercharge innovation—with unprecedented opportunities for growth and exploration. We believe the latest generation of Llama will ignite new applications and modeling paradigms, including synthetic data generation to enable the improvement and training of smaller models, as well as model distillation—a capability that has never been achieved at this scale in open source.
As part of this latest release, we’re introducing upgraded versions of the 8B and 70B models. These are multilingual and have a significantly longer context length of 128K, state-of-the-art tool use, and overall stronger reasoning capabilities. This enables our latest models to support advanced use cases, such as long-form text summarization, multilingual conversational agents, and coding assistants. We’ve also made changes to our license, allowing developers to use the outputs from Llama models—including the 405B—to improve other models. True to our commitment to open source, starting today, we’re making these models available to the community for download on llama.meta.com and Hugging Face and available for immediate development on our broad ecosystem of partner platforms. Model evaluations
For this release, we evaluated performance on over 150 benchmark datasets that span a wide range of languages. In addition, we performed extensive human evaluations that compare Llama 3.1 with competing models in real-world scenarios. Our experimental evaluation suggests that our flagship model is competitive with leading foundation models across a range of tasks, including GPT-4, GPT-4o, and Claude 3.5 Sonnet. Additionally, our smaller models are competitive with closed and open models that have a similar number of parameters.
Model Architecture As our largest model yet, training Llama 3.1 405B on over 15 trillion tokens was a major challenge. To enable training runs at this scale and achieve the results we have in a reasonable amount of time, we significantly optimized our full training stack and pushed our model training to over 16 thousand H100 GPUs, making the 405B the first Llama model trained at this scale.
To address this, we made design choices that focus on keeping the model development process scalable and straightforward. We opted for a standard decoder-only transformer model architecture with minor adaptations rather than a mixture-of-experts model to maximize training stability.
We adopted an iterative post-training procedure, where each round uses supervised fine-tuning and direct preference optimization. This enabled us to create the highest quality synthetic data for each round and improve each capability’s performance.
Compared to previous versions of Llama, we improved both the quantity and quality of the data we use for pre- and post-training. These improvements include the development of more careful pre-processing and curation pipelines for pre-training data, the development of more rigorous quality assurance, and filtering approaches for post-training data.
"""

In [21]:
QUESTION = "How many parameters LLaMA 3.1 model has?"

# Formulating the system prompt
system_prompt = (
    "You are an assistant and expert in answering questions from a chunks of content. "
    "Only answer AI-related question, else say that you cannot answer this question."
)
# Combining the system prompt with the user's question
prompt = (
    "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
    "Please provide an informative and accurate answer to the following question based on the avaiable context. Be concise and take your time. \nQuestion: {}\nAnswer:"
)
prompt = prompt.format(Example_chunk, QUESTION)

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", system_instruction=system_prompt
)

# Gemini API call
result = model.generate_content(
    prompt,
    request_options={"timeout": 1000},
)
res = result.text
print(res)

The provided text mentions a 405B parameter LLaMA 3.1 model, as well as upgraded 8B and 70B parameter models.



# Without Augmentation


Test the Gemini API to answer the same question without the addition of retrieved documents. Basically, the LLM will use its knowledge to answer the question.


In [22]:
QUESTION = "How many parameters LLaMA 3 model has?"

# Formulating the system prompt
system_prompt = "You are an assistant and expert in answering questions."

# Combining the system prompt with the user's question
prompt = "Be concise and take your time to answer the following question. \nQuestion: {}\nAnswer:"
prompt = prompt.format(QUESTION)

model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", system_instruction=system_prompt
)

# Gemini API call
result = model.generate_content(
    prompt,
    request_options={"timeout": 1000},
)
res = result.text

In [23]:
print(res)

The exact number of parameters for LLaMA 3 models isn't publicly available.  Meta has not released precise parameter counts for its various LLaMA 3 sizes.

